In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


### 1.Create a list with the motion verbs in 1QIsaa

#### 1.1 List of wanted motion verbs

In [24]:
motion_verbs = ['BW>[','HLK[','JY>[','JRD[','<BR[',
                '<LH[','CWB[','>TH[','BRX[','GJX[',
                'GLH[','GLL[','DXP[','DLG[','HWH=[',
                'XWC[','XLP[','XSH[','VB<[','VWF[',
                'MHR[','MWC[','NGC[','NHR[','NWX[',
                'NWS[','NXT[','NVP[','NS<[','NPL[',
                'NTK[','SBB[','SWR[','SLQ[','<WZ[',
                '<WP[','PNH[','PF<[','YWP[','Y<D[',
                'QHL[','QPY[','QRB[','RWY[','FVH[',
                'CWX[','CWR=[','CVP[','CQQ[','T>R[',
                'T<H[']

#### 1.2 List of occurrences of the motion verbs in 1QIsaa

In [25]:
verbs = TFOb.all("scroll", DSS).to_scrolls.filter(scroll="1Q8").to_words.filter_in(lex=motion_verbs)
verbs

<word_213 "JBW> TBW JB>W B>JM JBW> B> JPWL JRD J<LH HBJ>W TGLJ T<W <BRW NPL B> HT<W HT<W HT<WT CBW B> [...] MWLJKM TRD TNJXNW TT<NW CWB JRDT JRDT T<LJNH M<LH >BJ> JBW> CWVP JBW> HCJB B>H B>W HBJ>W JBJ>W JBW> JY>W">

In [26]:
#set(verbs.book)

In [27]:
verb = verbs[2]
print(verb)
scroll = verb.to_scrolls.scroll[0]
verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
print(verb.book[0], verb.chapter[0], verb.verse[0])
print(verse)
print(verse.to_signs.rec)

JB>W
Isaiah 13 2
<L HR NCPH F>W NS HRJMW QWL LHM HNJPW JD W JB>W PTXJ NDJBJM
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [63]:
word = TFOb(1898673, DSS)
signs = word.to_signs

print(word.g_cons)
print(signs.ids)
print(signs.glyphe)
print(signs.type)

[None]
[796615, 796616]
['', ' ? ']
['empty', 'unc']


In [64]:
sign

<sign_1 "">

In [9]:
def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [10]:
print(sign_info(verb))
print(verse)
print(len(sign_info(verb)), len(str(verse.to_signs)))

11 11 1111 110 00 00000 000 000 00011 11 1 1111 1111 111111
<L HR NCPH F>W NS HRJMW QWL LHM HNJPW JD W JB>W PTXJ NDJBJM
59 59


In [60]:
for v in verbs:
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([v.book[0], v.chapter[0], v.verse[0]], DSS, scroll)
    print(verse.ids)
    print(TFOb.section([v.book[0], v.chapter[0], v.verse[0]], DSS, scroll))
    print(sign_info(v))
    print(len(sign_info(v)), len(str(verse.to_signs)))
    print(len(sign_info(v)) == len(str(verse.to_signs)))
    print("\n")

[1895058, 1895059, 1895060, 1895061, 1895062, 1895063, 1895064, 1895065, 1895066, 1895067, 1895068, 1895069, 1895070, 1895071]
KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS XYRJ
000 0000 0 0000 000 00 000 0000 0 0000 0 0000 0000
50 50
True


[1895072, 1895073, 1895074, 1895075, 1895076, 1895077, 1895078, 1895079, 1895080, 1895081, 1895082, 1895083, 1895084, 1895085, 1895086, 1895087, 1895088, 1895089, 1895090, 1895091, 1895092]
LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ XWDC W CBT QR> MQR> LW> >WKL >WN W <YRTH
000 000000 0 0000 0000 000 0000 00000 000 00 0000 0 000 000 0000 000 0000 000 0 00000
85 85
True


[1895125, 1895126, 1895127, 1895128, 1895129, 1895130, 1895131, 1895132, 1895133, 1895134, 1895135, 1895136, 1895137]
RXYW W HZKW W HSJRW RW< M<LLJKM M NGD <JNJ XDLW HR<
0000 0 0000 0 00000 000 0000000 0 000 0000 0000 000
51 51
True


[1895149, 1895150, 1895151, 1895152, 1895153, 1895154, 1895155, 1895156, 1895157, 1895158, 1895159, 1895160, 1895161, 1895162, 1895163, 1895164, 189

86 86
True


[1897091, 1897092, 1897093, 1897094, 1897095, 1897096, 1897097, 1897098, 1897099, 1897100, 1897101, 1897102, 1897103, 1897104, 1897105, 1897106, 1897107, 1897108, 1897109, 1897110, 1897111, 1897112, 1897113, 1897114, 1897115, 1897116, 1897117, 1897118, 1897119, 1897120, 1897121, 1897122]
W JHJ B JMJ >XZ BN JWTM BN <WZJH MLK JHWDH <LH RYJN MLK >RM W PQX BN RWMLJH MLK JFR>L JRWCLM L MLXMH <LJH W LW> JKLW L HLXM <LJH
0 000 0 000 000 00 0000 00 00000 000 00000 000 0000 000 000 0 000 00 000000 000 00000 000000 0 00000 0000 0 000 0000 0 0000 0000
128 128
True


[1897148, 1897149, 1897150, 1897151, 1897152, 1897153, 1897154, 1897155, 1897156, 1897157, 1897158, 1897159, 1897160, 1897161, 1897162, 1897163, 1897164, 1897165, 1897166, 1897167, 1897168, 1897169, 1897170, 1897171, 1897172, 1897173, 1897174]
W JW>MR JHWH >L JC<JH Y> N> L QR>T >XZ >TH W C>R JCWB BNK >L QYH T<LT H BRKH H <LJWNH >L MSLT FDH KWBS
0 00000 0000 00 00000 00 00 0 0000 000 000 0 000 0000 000 00 000 0000 0 0000 0

39 39
True


[1899572, 1899573, 1899574, 1899575, 1899576, 1899577, 1899578, 1899579, 1899580, 1899581, 1899582, 1899583, 1899584, 1899585, 1899586, 1899587, 1899588, 1899589, 1899590, 1899591]
HNH JWM JHWH B> >GZRJ W <BRH W XRWN >P L FWM >RY L CMH W XV>JM JCMJD MMNH
000 000 0000 00 00000 0 0000 0 0000 00 0 000 000 0 000 0 00000 00000 0000
73 73
True


[1899592, 1899593, 1899594, 1899595, 1899596, 1899597, 1899598, 1899599, 1899600, 1899601, 1899602, 1899603, 1899604, 1899605, 1899606, 1899607, 1899608, 1899609, 1899610, 1899611]
KJ KWKBJ H CMJM W KSLJHM LW> J>JRW >WRM XCK H CMC B Y>TW W JRX LW> JGJH >WRW
00 00000 0 0000 0 000000 000 00000 0000 000 0 000 0 0000 0 000 000 0000 0000
76 76
True


[1899660, 1899661, 1899662, 1899663, 1899664, 1899665, 1899666, 1899667, 1899668, 1899669, 1899670, 1899671, 1899672, 1899673, 1899674, 1899675, 1899676, 1899677, 1899678, 1899679, 1899680]
W HJW K YBJ MDX W K Y>WN W >JN MQBY >JC >L <MW JPNW W >JC >L >RYW JNWSW
0 000 0 000 000 0 0 0000 0 000 0000

True


[1901981, 1901982, 1901983, 1901984, 1901985, 1901986, 1901987, 1901988, 1901989, 1901990, 1901991, 1901992, 1901993]
>MR CWMR >TH BWQR W GM LJLH >M TB<WN B<W CWBW >TJW
000 0000 000 0000 0 00 0000 00 00000 000 0000 0000
50 50
True


[1901981, 1901982, 1901983, 1901984, 1901985, 1901986, 1901987, 1901988, 1901989, 1901990, 1901991, 1901992, 1901993]
>MR CWMR >TH BWQR W GM LJLH >M TB<WN B<W CWBW >TJW
000 0000 000 0000 0 00 0000 00 00000 000 0000 0000
50 50
True


[1902005, 1902006, 1902007, 1902008, 1902009, 1902010, 1902011, 1902012, 1902013, 1902014, 1902015, 1902016, 1902017]
L QR>T YM> H>TJW MJM JWCBJ >RY TJM> B LXM QDMW NWDD
0 0000 000 00000 000 00000 000 0000 0 000 0000 0000
51 51
True


[1902068, 1902069, 1902070, 1902071, 1902072, 1902073, 1902074, 1902075, 1902076, 1902077, 1902078, 1902079]
MF> GJ XZWWN M LKJ >PW> KJ <LJTJ KWLK L GGWT
000 00 00000 0 000 0000 00 00000 0000 0 0000
44 44
True


[1902095, 1902096, 1902097, 1902098, 1902099, 1902100, 1902101, 1902102, 1902103

True


[1904474, 1904475, 1904476, 1904477, 1904478, 1904479, 1904480, 1904481, 1904482, 1904483, 1904484, 1904485, 1904486]
GM ZW>T M <M JHWH YB>WT JY>H HPLH <YH W HGDJL TWCJH
00 0000 0 00 0000 00000 0000 0000 000 0 00000 00000
51 51
True


[1904552, 1904553, 1904554, 1904555, 1904556, 1904557, 1904558, 1904559, 1904560, 1904561, 1904562, 1904563, 1904564, 1904565, 1904566, 1904567, 1904568, 1904569, 1904570]
W HJH K >BQ DQ HMWN ZDJK W K MWY <WBR HMWN <RJYJM W HJH L PT< PT>M
0 000 0 000 00 0000 0000 0 0 000 0000 0000 000000 0 000 0 000 0000
66 66
True


[1904749, 1904750, 1904751, 1904752, 1904753, 1904754, 1904755, 1904756, 1904757, 1904758, 1904759, 1904760, 1904761, 1904762, 1904763, 1904764, 1904765, 1904766, 1904767, 1904768, 1904769, 1904770, 1904771, 1904772, 1904773, 1904774, 1904775, 1904776, 1904777]
W JW>MR >DWNJ J<N KJ NGC H <M H ZH B PJW W B FPTJW KBDWNJ W LBW RXWQ MMNJ W THJH JR>T >WTJ K MYWT >NCJM MLMDH
0 00000 00000 000 00 000 0 00 0 00 0 000 0 0 00000 000000 0 000 000

...

102 102
True


[1910410, 1910411, 1910412, 1910413, 1910414, 1910415, 1910416, 1910417, 1910418, 1910419, 1910420, 1910421, 1910422, 1910423, 1910424, 1910425, 1910426, 1910427, 1910428, 1910429, 1910430, 1910431, 1910432, 1910433, 1910434, 1910435, 1910436, 1910437]
MJ> NTN L MCWSH J<QWB W JFR>L L BWZZJM H LW> JHWH ZH XV>NW LW W LW> >BW B DRKJW L HLWK W LW> CM<W B TWRTJW
000 000 0 00000 00000 0 00000 0 000000 0 000 0000 00 00000 00 0 000 000 0 00000 0 0000 0 000 0000 0 000000
106 106
True


[1910482, 1910483, 1910484, 1910485, 1910486, 1910487, 1910488, 1910489, 1910490, 1910491, 1910492, 1910493, 1910494, 1910495, 1910496, 1910497, 1910498, 1910499, 1910500, 1910501, 1910502, 1910503, 1910504]
KJ> T<BWR B MJM >TKH >NJ W B NHRWT LW> JCVPWK KJ> TLK BMW >C LW> TKWH W LHBH LW> TB<R BKH
000 00000 0 000 0000 000 0 0 00000 000 000000 000 000 000 00 000 0000 0 0000 000 0000 000
89 89
True


[1910482, 1910483, 1910484, 1910485, 1910486, 1910487, 1910488, 1910489, 1910490, 1910491, 1910492, 19

61 61
True


[1912427, 1912428, 1912429, 1912430, 1912431, 1912432, 1912433, 1912434, 1912435, 1912436, 1912437, 1912438, 1912439, 1912440, 1912441, 1912442, 1912443, 1912444, 1912445, 1912446, 1912447, 1912448, 1912449, 1912450, 1912451, 1912452, 1912453, 1912454, 1912455, 1912456, 1912457, 1912458]
CM<W ZW>T BJT J<QWB H NQR>JM B CM JFR>L W M MJ JHWDH JY>W H NCB<JM B CM JHWH W B >LWHJ JFR>L JZKJRW LW> B >MT W LW> B YDQH
0000 0000 000 00000 0 000000 0 00 00000 0 0 00 00000 0000 0 000000 0 00 0000 0 0 00000 00000 000000 000 0 000 0 000 0 0000
122 122
True


[1912474, 1912475, 1912476, 1912477, 1912478, 1912479, 1912480, 1912481, 1912482, 1912483, 1912484, 1912485, 1912486, 1912487, 1912488, 1912489]
H RJCWNWT M >Z HGDTJ W M PJ JY>H W >CMJ<M PT>WM <FJTJ W TBW>JNH
0 0000000 0 00 00000 0 0 00 0000 0 000000 00000 00000 0 0000000
63 63
True


[1912474, 1912475, 1912476, 1912477, 1912478, 1912479, 1912480, 1912481, 1912482, 1912483, 1912484, 1912485, 1912486, 1912487, 1912488, 1912489]
H RJCW

95 95
True


[1914700, 1914701, 1914702, 1914703, 1914704, 1914705, 1914706, 1914707, 1914708, 1914709, 1914710, 1914711, 1914712, 1914713, 1914714, 1914715, 1914716, 1914717, 1914718, 1914719, 1914720, 1914721, 1914722]
KJ> H HRJM JMWCW W H GB<WT TTMWVJNH W XSDJ M >TK LW> JMWC W BRJT CLWMJ LW> TMWV >MR MRXMKJ JHWH
000 0 0000 00000 0 0 00000 00000000 0 0000 0 000 000 0000 0 0000 00000 000 0000 000 000000 0000
95 95
True


[1914764, 1914765, 1914766, 1914767, 1914768, 1914769, 1914770, 1914771, 1914772, 1914773, 1914774, 1914775, 1914776, 1914777, 1914778, 1914779, 1914780]
B YDQH TTKWNNJ RXQJ M <WCQ KJ> LW> TJR>J W M MXTH KJ> LW> TQRB >LJK
0 0000 0000000 0000 0 0000 000 000 00000 0 0 0000 000 000 0000 0000
67 67
True


[1914781, 1914782, 1914783, 1914784, 1914785, 1914786, 1914787, 1914788, 1914789, 1914790, 1914791, 1914792]
HNH GWR JGWR >KS M >TJ MJ JGR >TK <LJK JPWLW
000 000 0000 000 0 000 00 000 000 0000 00000
44 44
True


[1914793, 1914794, 1914795, 1914796, 1914797, 1914798, 1914

True


[1916392, 1916393, 1916394, 1916395, 1916396, 1916397, 1916398, 1916399, 1916400, 1916401, 1916402, 1916403, 1916404, 1916405, 1916406, 1916407, 1916408, 1916409, 1916410, 1916411, 1916412, 1916413, 1916414]
W JJR>W M M<RB >T CM JHWH W M MZRX CMC >T KBWDJW KJ> JBW> K NHR YWR RWX JHWH NWSSH BWH
0 00000 0 0000 00 00 0000 0 0 0000 000 00 000000 000 0000 0 000 000 000 0000 00000 000
86 86
True


[1916392, 1916393, 1916394, 1916395, 1916396, 1916397, 1916398, 1916399, 1916400, 1916401, 1916402, 1916403, 1916404, 1916405, 1916406, 1916407, 1916408, 1916409, 1916410, 1916411, 1916412, 1916413, 1916414]
W JJR>W M M<RB >T CM JHWH W M MZRX CMC >T KBWDJW KJ> JBW> K NHR YWR RWX JHWH NWSSH BWH
0 00000 0 0000 00 00 0000 0 0 0000 000 00 000000 000 0000 0 000 000 000 0000 00000 000
86 86
True


[1916415, 1916416, 1916417, 1916418, 1916419, 1916420, 1916421, 1916422, 1916423, 1916424, 1916425, 1916426, 1916427, 1916428]
W B> >L YJWN GW>L W L CBJ PC< B J<QWB NW>M JHWH
0 00 00 0000 0000 0 0 000 00

True


[1918888, 1918889, 1918890, 1918891, 1918892, 1918893, 1918894, 1918895, 1918896, 1918897, 1918898, 1918899, 1918900, 1918901, 1918902, 1918903, 1918904, 1918905, 1918906, 1918907, 1918908, 1918909, 1918910, 1918911]
W HJH M DJ XWDC B XWDCW W M DJ CBT B CBTH JBW> KWL H BFR L HCTXWT L PNJ >MR JHWH
0 000 0 00 0000 0 00000 0 0 00 000 0 0000 0000 000 0 000 0 000000 0 000 000 0000
80 80
True


[1918912, 1918913, 1918914, 1918915, 1918916, 1918917, 1918918, 1918919, 1918920, 1918921, 1918922, 1918923, 1918924, 1918925, 1918926, 1918927, 1918928, 1918929, 1918930, 1918931, 1918932, 1918933, 1918934, 1918935, 1918936, 1918937, 1918938]
W JY>W W R>W B PGRJ H >NCJM H PWC<JM BJ> KJ> TWL<TM LW> TMWT W >CHMH LW> TKBH W HJW DR>WN L KWL H BFR
0 0000 0 000 0 0000 0 00000 0 000000 000 000 000000 000 0000 0 01000 000 0000 0 000 00000 0 000 0 000
101 101
True




In [13]:
# change the sign info column on a dataset
df = pd.read_csv("data/isaiah_with_loca_2024-05-22_10_32.csv").fillna("")

In [14]:
df

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,...,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,comments
0,212256,BW>[,MT,Isaiah,1,12,TB>W,KJ TB>W L R>WT PNJ MJ BQC Z>T M JDKM RMS XYRJ,,qal,...,,,,,,,,,,
1,1895059,BW>[,1Qisaa,Isaiah,1,12,TB>W,KJ> TB>W L R>WT PNJ MJ BQC ZW>T M JDKM L RMWS ...,000 0000 0 0000 0000 00 000 0000 0 00000 0 000...,qal,...,,,,,,,,,,
2,212269,BW>[,MT,Isaiah,1,13,HBJ>,L> TWSJPW HBJ> MNXT CW> QVRT TW<BH HJ> LJ XDC ...,,hif,...,,,,,,,,,,
3,1895075,BW>[,1Qisaa,Isaiah,1,13,HBJ>,LW> TWSJPW L HBJ> MNXT CW> QVRT TW<BH HJ> LJ X...,000 000000 0 0000 0000 000 0000 00000 000 000 ...,hifil,...,,,,,,,,,,
4,212315,SWR[,MT,Isaiah,1,16,HSJRW,RXYW HZKW HSJRW R< M<LLJKM M NGD <JNJ XDLW HR<,,hif,...,subs,complex,fictive,MN,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,1927678,BW>[,1Q8,Isaiah,66,23,JBW>,W HJH M DJ XDC B XDCW W M DJ CBT B CBTW JBW> K...,0 000 0 00 000 0 00000 0 0 00 000 0 00000 0000...,qal,...,,,,,,,,,,
2196,234980,JY>[,MT,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PC<JM BJ KJ TWL<...,,qal,...,,,,,,,,,,
2197,1918913,JY>[,1Qisaa,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PWC<JM BJ> KJ> T...,0 0000 0 000 0 0000 0 00000 0 000000 0000 000 ...,qal,...,,,,,,,,,,
2198,1918913,JY>[,1Qisaa,Isaiah,66,24,JY>W,W JY>W W R>W B PGRJ H >NCJM H PWC<JM BJ> KJ> T...,0 0000 0 000 0 0000 0 00000 0 000000 000 000 0...,qal,...,,,,,,,,,,


In [16]:
def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [19]:
df[df.scroll == "1Q8"]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,...,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,comments
171,1919017,BW>[,1Q8,Isaiah,7,24,JBW>,B XYJM W B QCT JBW> CMH KJ CMJR W CJT THJH KL ...,1 1111 0 0 000 0000 000 00 1111 1 111 1111 11 ...,qal,...,adv,simple,factive,,,,,,,
175,1919037,BW>[,1Q8,Isaiah,7,25,TBW,W KL H HRJM >CR B M<DR J<DRWN L> TBW CMH JR>T ...,1 11 1 1111 111 1 1111 000000 01 111 000 1111 ...,qal,...,adv,simple,factive,,,,,,,
301,1919255,BW>[,1Q8,Isaiah,13,2,JB>W,<L HR NCPH F>W NS HRJMW QWL LHM HNJPW JD W JB>...,11 11 1111 110 00 00000 000 000 00011 11 1 111...,qal,...,subs,complex,factive,,,,,,,
306,1919289,BW>[,1Q8,Isaiah,13,5,B>JM,B>JM M >RY MRXQ M QYH H CMJM JHWH W KLJ Z<MW L...,1111 1 111 1111 1 111 1 1110 1110 0 000 0000 0...,qal,...,subs,complex,factive,MN,MN,,,,,
309,1919316,BW>[,1Q8,Isaiah,13,6,JBW>,HJLJLW KJ QRWB JWM JHWH K CD M CDJ JBW>,111111 11 1111 111 1111 1 11 1 111 1111,qal,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,1927616,BW>[,1Q8,Isaiah,66,20,JBJ>W,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,0 00000 00 01 11100 0 00 1 1111 1111 1 1110 0 ...,hifil,...,subs,complex,factive,,,,,,,
2190,1927583,BW>[,1Q8,Isaiah,66,20,HBJ>W,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,0 00000 00 01 11100 0 00 1 1111 1111 1 1110 0 ...,hif,...,,,,,,,,,,
2191,1927616,BW>[,1Q8,Isaiah,66,20,JBJ>W,W HBJ>W >T KL >XJKM M KL H GWJM MNXH L JHWH B ...,0 00000 00 01 11100 0 00 1 1111 1111 1 1110 0 ...,hif,...,,,,,,,,,,
2195,1927678,BW>[,1Q8,Isaiah,66,23,JBW>,W HJH M DJ XDC B XDCW W M DJ CBT B CBTW JBW> K...,1 111 1 00 001 1 1000 0 0 00 110 0 0000 0000 0...,qal,...,,,,,,,,,,


In [21]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [23]:
df.to_csv("data/isaiah_with_loca.csv", index=False)